Dataset Preparation: you have to break you data into train test split

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

def split_images(
    input_dir,
    output_dir,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    seed=42
):
    assert round(train_ratio + val_ratio + test_ratio, 2) == 1.0, "Ratios must sum to 1.0"

    class_names = os.listdir(input_dir)

    for class_name in class_names:
        class_dir = os.path.join(input_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        images = os.listdir(class_dir)
        images = [img for img in images if img.lower().endswith(('.jpg', '.jpeg', '.png'))]

        train_imgs, temp_imgs = train_test_split(images, train_size=train_ratio, random_state=seed)
        val_imgs, test_imgs = train_test_split(temp_imgs, test_size=test_ratio / (val_ratio + test_ratio), random_state=seed)

        for split, split_imgs in zip(['train', 'validation', 'test'], [train_imgs, val_imgs, test_imgs]):
            split_class_dir = os.path.join(output_dir, split, class_name)
            os.makedirs(split_class_dir, exist_ok=True)

            for img in split_imgs:
                src_path = os.path.join(class_dir, img)
                dst_path = os.path.join(split_class_dir, img)
                shutil.copy2(src_path, dst_path)

    print(f"Dataset split into train/validation/test at {output_dir}")

split_images(
    input_dir="banana_dataset",        
    output_dir="output_banana_dataset",                
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15
)


Dataset split into train/validation/test at output_banana_dataset


Importing the pretrained model:
you have to check your dataset with different model to get better result

ResNet50


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x) 

num_classes = 7 
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

InceptionV3

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

num_classes = 7 

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x) 

predictions = Dense(num_classes, activation='softmax')(x) 

model = Model(inputs=base_model.input, outputs=predictions)

MobileNet

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x) 

predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

Model Training: you have to write this piece of code multiple time because each model accept the diffrent dimension

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Data/fine_output_banana_dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical' 
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/Data/fine_output_banana_dataset/validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Data/fine_output_banana_dataset/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False 
)


Found 895 images belonging to 7 classes.
Found 189 images belonging to 7 classes.
Found 196 images belonging to 7 classes.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Data/fine_output_banana_dataset/train',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical' 
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/Data/fine_output_banana_dataset/validation',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Data/fine_output_banana_dataset/test',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False 
)


Found 895 images belonging to 7 classes.
Found 189 images belonging to 7 classes.
Found 196 images belonging to 7 classes.


Compiling the model

In [7]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Training the model with the dataset

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_banana_freshness_model.h5',
                                   monitor='val_accuracy',
                                   save_best_only=True,
                                   mode='max',
                                   verbose=1)

history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=[early_stopping, model_checkpoint]
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 13s/step - accuracy: 0.2699 - loss: 1.8238 
Epoch 1: val_accuracy improved from -inf to 0.57672, saving model to best_banana_freshness_model.h5


28/28 ━━━━━━━━━━━━━━━━━━━━ 519s 18s/step - accuracy: 0.2747 - loss: 1.8145 - val_accuracy: 0.5767 - val_loss: 1.1531
Epoch 2/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 842ms/step - accuracy: 0.5879 - loss: 1.0951
Epoch 2: val_accuracy improved from 0.57672 to 0.69841, saving model to best_banana_freshness_model.h5


28/28 ━━━━━━━━━━━━━━━━━━━━ 25s 910ms/step - accuracy: 0.5890 - loss: 1.0937 - val_accuracy: 0.6984 - val_loss: 0.9228
Epoch 3/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 848ms/step - accuracy: 0.7235 - loss: 0.8912
Epoch 3: val_accuracy improved from 0.69841 to 0.71429, saving model to best_banana_freshness_model.h5


28/28 ━━━━━━━━━━━━━━━━━━━━ 26s 917ms/step - accuracy: 0.7232 - loss: 0.8905 - val_accuracy: 0.7143 - val_loss: 0.8136
Epoch 4/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 852ms/step - accuracy: 0.7391 - loss: 0.8141
Epoch 4: val_accuracy improved from 0.71429 to 0.79365, saving model to best_banana_freshness_model.h5


...

28/28 ━━━━━━━━━━━━━━━━━━━━ 25s 907ms/step - accuracy: 0.8469 - loss: 0.4992 - val_accuracy: 0.8783 - val_loss: 0.4706
Epoch 11/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 851ms/step - accuracy: 0.8575 - loss: 0.5099
Epoch 11: val_accuracy improved from 0.87831 to 0.89947, saving model to best_banana_freshness_model.h5


28/28 ━━━━━━━━━━━━━━━━━━━━ 26s 926ms/step - accuracy: 0.8580 - loss: 0.5086 - val_accuracy: 0.8995 - val_loss: 0.4408
Epoch 12/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 838ms/step - accuracy: 0.8637 - loss: 0.4799
Epoch 12: val_accuracy did not improve from 0.89947
28/28 ━━━━━━━━━━━━━━━━━━━━ 25s 884ms/step - accuracy: 0.8640 - loss: 0.4791 - val_accuracy: 0.8942 - val_loss: 0.4304
Epoch 13/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 789ms/step - accuracy: 0.8897 - loss: 0.4239
Epoch 13: val_accuracy improved from 0.89947 to 0.90476, saving model to best_banana_freshness_model.h5


28/28 ━━━━━━━━━━━━━━━━━━━━ 24s 873ms/step - accuracy: 0.8893 - loss: 0.4241 - val_accuracy: 0.9048 - val_loss: 0.4270
Epoch 14/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 831ms/step - accuracy: 0.8495 - loss: 0.4656
Epoch 14: val_accuracy did not improve from 0.90476
28/28 ━━━━━━━━━━━━━━━━━━━━ 25s 876ms/step - accuracy: 0.8502 - loss: 0.4645 - val_accuracy: 0.8466 - val_loss: 0.4455
Epoch 15/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 834ms/step - accuracy: 0.8558 - loss: 0.4450
Epoch 15: val_accuracy did not improve from 0.90476
28/28 ━━━━━━━━━━━━━━━━━━━━ 25s 879ms/step - accuracy: 0.8557 - loss: 0.4450 - val_accuracy: 0.9048 - val_loss: 0.3993
Epoch 16/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 850ms/step - accuracy: 0.8890 - loss: 0.3749
Epoch 16: val_accuracy did not improve from 0.90476
28/28 ━━━━━━━━━━━━━━━━━━━━ 25s 904ms/step - accuracy: 0.8888 - loss: 0.3754 - val_accuracy: 0.8942 - val_loss: 0.3775
Epoch 17/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 828ms/step - accuracy: 0.8943 - loss: 0.3606
Epoch 17: val_accuracy d

28/28 ━━━━━━━━━━━━━━━━━━━━ 25s 894ms/step - accuracy: 0.8889 - loss: 0.3617 - val_accuracy: 0.9259 - val_loss: 0.3134
Epoch 22/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 830ms/step - accuracy: 0.9041 - loss: 0.3214
Epoch 22: val_accuracy did not improve from 0.92593
28/28 ━━━━━━━━━━━━━━━━━━━━ 24s 874ms/step - accuracy: 0.9040 - loss: 0.3217 - val_accuracy: 0.9206 - val_loss: 0.3095
Epoch 23/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 836ms/step - accuracy: 0.9094 - loss: 0.3239
Epoch 23: val_accuracy improved from 0.92593 to 0.93651, saving model to best_banana_freshness_model.h5


28/28 ━━━━━━━━━━━━━━━━━━━━ 25s 903ms/step - accuracy: 0.9096 - loss: 0.3236 - val_accuracy: 0.9365 - val_loss: 0.3111
Epoch 24/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 783ms/step - accuracy: 0.8997 - loss: 0.3457
Epoch 24: val_accuracy did not improve from 0.93651
28/28 ━━━━━━━━━━━━━━━━━━━━ 23s 837ms/step - accuracy: 0.8996 - loss: 0.3457 - val_accuracy: 0.9206 - val_loss: 0.2927
Epoch 25/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 811ms/step - accuracy: 0.8973 - loss: 0.3348
Epoch 25: val_accuracy did not improve from 0.93651
28/28 ━━━━━━━━━━━━━━━━━━━━ 41s 850ms/step - accuracy: 0.8973 - loss: 0.3350 - val_accuracy: 0.9153 - val_loss: 0.3080
Epoch 26/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 769ms/step - accuracy: 0.8854 - loss: 0.3443
Epoch 26: val_accuracy did not improve from 0.93651
28/28 ━━━━━━━━━━━━━━━━━━━━ 23s 823ms/step - accuracy: 0.8858 - loss: 0.3436 - val_accuracy: 0.9365 - val_loss: 0.2877
Epoch 27/30
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 809ms/step - accuracy: 0.9172 - loss: 0.3351
Epoch 27: val_accuracy d

Model evaluation

In [9]:
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 88s 15s/step - accuracy: 0.8999 - loss: 0.3131
Test Loss: 0.2619
Test Accuracy: 0.9133


Getting the detailed metrics of the model

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

test_labels = test_generator.classes
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

class_names = list(train_generator.class_indices.keys())

idx_to_class = {v: k for k, v in train_generator.class_indices.items()}
sorted_class_names = [idx_to_class[i] for i in sorted(idx_to_class.keys())]


print("Confusion Matrix:")
print(confusion_matrix(test_labels, predicted_classes))

print("\nClassification Report:")
print(classification_report(test_labels, predicted_classes, target_names=sorted_class_names))

7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 729ms/step
Confusion Matrix:
[[ 0 24  0  0  4  0  0]
 [ 0 22  0  0  6  0  0]
 [ 0 17  0  0 11  0  0]
 [ 1  3  0  0 24  0  0]
 [ 0  3  0  0 19  0  6]
 [ 0  1  0  0  8  0 19]
 [ 0  0  0  0  3  0 25]]

Classification Report:
              precision    recall  f1-score   support

1_days_older       0.00      0.00      0.00        28
2_days_older       0.31      0.79      0.45        28
3_days_older       0.00      0.00      0.00        28
4_days_older       0.00      0.00      0.00        28
5_days_older       0.25      0.68      0.37        28
6_days_older       0.00      0.00      0.00        28
7_days_older       0.50      0.89      0.64        28

    accuracy                           0.34       196
   macro avg       0.15      0.34      0.21       196
weighted avg       0.15      0.34      0.21       196



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Saving the model

In [8]:
model.save('final_banana_freshness_model.h5')

Loading the model

In [10]:
from tensorflow.keras.models import load_model
loaded_model = load_model('best_banana_freshness_model.h5')

Predicting using the model

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_freshness(image_path, model, class_names):
    img = image.load_img(image_path, target_size=(299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) 
    img_array /= 255.0 

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_freshness = class_names[predicted_class_index]
    confidence = predictions[0][predicted_class_index]

    return predicted_freshness, confidence

class_names_list = list(train_generator.class_indices.keys())
idx_to_class_map = {v: k for k, v in train_generator.class_indices.items()}
sorted_class_names_for_prediction = [idx_to_class_map[i] for i in sorted(idx_to_class_map.keys())]


image_to_predict = '/content/Screenshot 2025-07-17 031004.png'
freshness, confidence = predict_freshness(image_to_predict, loaded_model, sorted_class_names_for_prediction)

print(f"The banana is predicted to be: {freshness} with {confidence*100:.2f}% confidence.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 33s 33s/step
The banana is predicted to be: 2_days_older with 52.42% confidence.


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()